## 이론 
- 연관규칙이란? <br>
A라는 상품과 B라는 상품이 서로 연관성이 있다고 하면 A->B 라고 하는 연관 규칙을 만들 수 있다. <br>
하지만 상품이 수백, 수천개라면 이러한 연관 규칙도 엄청나게 많이 생성되어서<br>
이 중에 과연 어떤 연관 규칙이 가장 크리티컬하게 사용될 수 있는지 평가가 필요하다. <br>
이러한 연관규칙 평가에는 3가지 척도가 쓰이는데, '지지도','신뢰도','향상도'라고 하는 척도이다

1. 지지도(Support) <br>
전체 거래항목 중 상품 A와 상품 B를 동시에 포함하여 거래하는 비율을 의미하는데,<br>
A->B 라고 하는 규칙이 전체 거래 중 차지하는 비율을 통해 해당 연관 규칙이 얼마나 의미가 있는 규칙인지를 확인합니다. <br>
<br>
**지지도 = P(A∩B) : (A와 B가 동시에 포함된 거래 수 / 전체 거래 수)**<br>
<br>


2. 신뢰도(Confidence)<br>
상품 A를 포함하는 거래 중 A와 B가 동시에 거래되는 비중으로,<br>
상품 A를 구매 했을 때 상품 B를 구매할 확률이 어느정도 되는지를 확인<br>
<br>
**신뢰도 = P(A∩B)/P(A) : (A와B가 동시에 포함된 거래 수 / A가 포함된 거래 수)** <br>
<br>


3. 향상도(Lift) <br>
상품 A의 거래 중 항목 B가 포함된 거래의 비율 / 전체 상품 거래 중 상품 B가 거래된 비율<br>
(A가 주어지지 않았을 때 B의 확률 대비 A가 주어졌을 때 B의 확률 증가 비율) <br>
<br>
**향상도 = P(B|A)/P(B) : A와B가 동시에 일어난 횟수 / A,B가 독립된 사건일 때 A,B가 동시에 일어날 확률**<br>
품목 A와 B사이에 아무런 관계가 상호 관계가 없으면 향상도는 1이고, 향상도가 1보다 높아질 수록 연관성이 높다고 할 수 있다.<br> 
이것은 또한 향상도가 1보다 크거나 작다면 우연적 기회(Random Chance)보다 우수하다고 해석
<br><br><br>

상품의 갯수가 증가될수록 계산량이 기하급수적으로 늘어나게 되므로 구매 데이터가 적은 상품은 과감히 버릴 수 있어야 하며, 현업에서 의미있게 사용될만한 연관 규칙을 의논하는 과정도 있어야 한다... 계산량이 많음,,

## 코드 convert 함
버전 안 맞아서..

In [8]:
from numpy import *

In [9]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [10]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    C1.sort()
    return list(map(frozenset, C1))# use frozen set so we can use it as a key in a dict    

In [11]:
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData

In [12]:
dataSet = loadDataSet()
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [13]:
# 후보 아이템 집합인 C1을 생성함
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [14]:
# D is a dataset in the setform about dataSet.
D = list(map(set,dataSet))
D

[{1, 3, 4}, {2, 3, 5}, {1, 2, 3, 5}, {2, 5}]

In [15]:
# 최소 지지도 0.5 만족 변수 추출
L1,suppDat0 = scanD(D,C1,0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [16]:
def aprioriGen(Lk, k): #creates Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #if first k-2 elements are equal
                retList.append(Lk[i] | Lk[j]) #set union
    return retList

In [17]:
def apriori(dataSet, minSupport = 0.5):
    C1 = createC1(dataSet)
    D = list(map(set, dataSet))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#scan DB to get Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData

In [30]:
# apriori 결과 확인 
# L은 빈발 아이템 집합에 대한 목록을 포함하고 있으며, 최소 지지도는 0.5이다. 
L,suppData = apriori(dataSet,0.5)
L

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [31]:
L,suppData = apriori(dataSet,0.7)
L

[[frozenset({5}), frozenset({2}), frozenset({3})], [frozenset({2, 5})], []]

In [19]:
L[0]

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [20]:
L[1]

[frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})]

In [21]:
L[2]

[frozenset({2, 3, 5})]

In [22]:
L[3]

[]

In [23]:
aprioriGen(L[0],2)

[frozenset({2, 5}),
 frozenset({3, 5}),
 frozenset({1, 5}),
 frozenset({2, 3}),
 frozenset({1, 2}),
 frozenset({1, 3})]

In [24]:
def generateRules(L, supportData, minConf=0.7):  #supportData is a dict coming from scanD
    bigRuleList = []
    for i in range(1, len(L)):#only get the sets with two or more items
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList   

In [25]:
def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    prunedH = [] #create new list to return
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq] #calc confidence
        if conf >= minConf: 
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

In [26]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)): #try further merging
        Hmp1 = aprioriGen(H, m+1)#create Hm+1 new candidates
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):    #need at least two sets to merge
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [35]:
dataSet

[[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [32]:
L,suppData= apriori(dataSet,minSupport=0.5)
L
# 지지도 0.5로 하는 빈발 아이템 집합을 생성
# 지지도 = A와 B가 동시에 포함된 거래 수 / 전체 거래 수

[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})],
 [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})],
 [frozenset({2, 3, 5})],
 []]

In [29]:
# 신뢰도 계산.. 상품 A를 구매 했을 때 상품 B를 구매할 확률이 어느정도 되는지를 확인
rules= generateRules(L,suppData, minConf=0.7)

frozenset({5}) --> frozenset({2}) conf: 1.0
frozenset({2}) --> frozenset({5}) conf: 1.0
frozenset({1}) --> frozenset({3}) conf: 1.0


## Example 수행

In [40]:
mushDatSet = [line.split() for line in open('mushroom.dat.txt').readlines()]
# list로 넣네... 

In [43]:
len(mushDatSet) # 행

8124

In [46]:
len(mushDatSet[0]) #열

23

In [49]:
L,suppData= apriori(mushDatSet, minSupport=0.3)
# 지지도 0.3이상 상품들

In [51]:
for item in L[1]:
    if item.intersection('2'):
        print (item)

frozenset({'28', '2'})
frozenset({'2', '53'})
frozenset({'2', '23'})
frozenset({'34', '2'})
frozenset({'2', '36'})
frozenset({'2', '59'})
frozenset({'63', '2'})
frozenset({'67', '2'})
frozenset({'76', '2'})
frozenset({'2', '85'})
frozenset({'2', '86'})
frozenset({'2', '90'})
frozenset({'2', '93'})
frozenset({'39', '2'})


In [52]:
for item in L[6]:
    if item.intersection('2'):
        print (item)

frozenset({'34', '90', '85', '2', '39', '53', '86'})
frozenset({'28', '85', '90', '2', '39', '53', '86'})
frozenset({'28', '34', '90', '2', '39', '53', '86'})
frozenset({'28', '34', '85', '2', '39', '53', '86'})
frozenset({'28', '34', '85', '90', '2', '39', '53'})
frozenset({'28', '34', '85', '90', '2', '39', '86'})
frozenset({'28', '34', '85', '90', '2', '53', '86'})
frozenset({'28', '34', '85', '2', '59', '39', '86'})
frozenset({'28', '34', '63', '85', '2', '39', '86'})
frozenset({'85', '93', '90', '23', '2', '59', '86'})
frozenset({'85', '93', '23', '2', '59', '39', '86'})
frozenset({'63', '36', '90', '93', '85', '2', '59'})
frozenset({'63', '85', '23', '2', '59', '39', '86'})
frozenset({'63', '36', '85', '2', '59', '39', '86'})
frozenset({'63', '36', '23', '2', '59', '39', '86'})
frozenset({'63', '36', '85', '93', '2', '59', '39'})
frozenset({'63', '36', '85', '23', '2', '39', '86'})
frozenset({'63', '36', '85', '23', '2', '59', '39'})
frozenset({'63', '36', '85', '23', '2', '59', 

In [56]:
rules= generateRules(L,suppData, minConf=0.7)
# 신뢰도 0.7이상
# frozenset({'1'}) --> frozenset({'24'}) conf: 0.8406537282941777
# 상품 "1"을 구매 했을 때 "24"를 구매할 확률 0.84

이거 실제 데이터 반영하면,,, 오래 걸릴 거 같은데.. 